<a href="https://colab.research.google.com/github/wynniegross1/vegetation_anomalies/blob/main/NDVI_over_time_modis_8day.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **NDVI Over Time for Region of Choice**


---


Original GEE Script Author: Aaron Eubank

Python Script Adaptation: Wynnie Gross


---

*   This script is run primarily to get a map of NDVI anomaly percentages of a certain period of time versus a baseline period.
*   The 8-day Modis data used in this analysis has about a 2-week lag, meaning that it's not ideal for up-to-the-minute results.
*   Methodology adapted from GEARS - Geospatial Ecology and Remote Sensing - https://www.geospatialecology.com

#### Importing Packages and Authenticating GEE API

In [ ]:
!pip install geemap
!pip install earthengine-api

In [20]:
####################################
## IMPORT PACKAGES/SET UP PROJECT ##
####################################

# Import necessary packages
import geemap
import ee
from google.colab import userdata
from datetime import datetime

# Initialize GEE Project and Authenticate
ee.Authenticate()
ee.Initialize(project = userdata.get('projectname')) # Put your own API Key in the "Secrets" Tab on the left side

#################################
### DEFINE REGION OF INTEREST ###
#################################

# Define the region of interest
# region_of_interest = ee.FeatureCollection("path/to/region_of_interest") # CHANGE TO SELECT YOUR CHOSEN REGION

# Select regions for analysis if applicable
# selected = region_of_interest.filter(ee.Filter.Or(
    # ee.Filter.eq('FIELD', 'FEATURE'), # CHANGE TO SELECT FIELD/FEATURE FOR YOUR REGION
    # ee.Filter.eq('FIELD', 'FEATURE')  # CHANGE TO SELECT FIELD/FEATURE FOR YOUR REGION
# ))

## EXAMPLE ##

region_of_interest = ee.FeatureCollection("projects/ee-aeubank/assets/sudan_admin1") # EXAMPLE

selected = region_of_interest.filter(ee.Filter.Or(
    ee.Filter.eq('admin_1', 'Sennar'),
    ee.Filter.eq('admin_1', 'Aj Jazirah')
))

region_of_interest = selected

# Create a map object
Map = geemap.Map(center=[14, 34], zoom=7) # CHANGE CENTER AND ZOOM LEVEL FOR YOUR CHOSEN ROI

# Add vector layer
Map.addLayer(region_of_interest, {'color': 'gray', 'strokeWidth': 1}, 'Region of Interest')

##########################################
###### LOAD IN DATA / FILTER DATES #######
##########################################

# Load MODIS 8-day data
modis8 = ee.ImageCollection('MODIS/061/MOD09A1')

# Filter for region of interest
modis8 = modis8.filterBounds(region_of_interest)

# Define the current date dynamically (using datetime)
now = datetime.now().isoformat()

########################################
###### CROPLAND AND CLOUD MASKS ########
########################################

## CROPLAND MASK ##

# Load in ESA WorldCover dataset
esa = ee.ImageCollection('ESA/WorldCover/v100')

# Get the latest ESA WorldCover image
esa_latest = esa.limit(1, 'system:time_start').first()

# Get only cropland for region of interest from ESA WorldCover image
cropland = esa_latest.updateMask(esa_latest.select('Map').eq(40).clip(region_of_interest))

# Create cropland mask function
def cropmask(image):
    return image.updateMask(cropland)

## CLOUD MASK ##

# Define function to mask out cloudy pixels
def maskClouds(image):
    # Select the QA band
    QA = image.select(['StateQA'])
    # Make a mask to get bit 10, the internal_cloud_algorithm_flag bit
    bitMask = 1 << 10
    # Return an image masking out cloudy areas
    return image.updateMask(QA.bitwiseAnd(bitMask).eq(0))


###########################################
############# CALCULATE NDVI ##############
###########################################

# Create NDVI function
def add_ndvi(image):
    # Calculate NDVI using the normalized difference between NIR and Red bands
    ndvi = image.normalizedDifference(['sur_refl_b02', 'sur_refl_b01']) \
                .multiply(10000).toInt16()  # Multiply by 10000 and cast to Int16 for better precision
    # Add the NDVI band to the image
    return image.addBands(ndvi.rename('ndvi'))

#################################################################
######## APPLYING NDVI / CLOUD / CROPLAND FUNCTIONS #############
#################################################################

# Filter dataset to admin bounds, dates, and apply cloud mask, calculate NDVI

# 2012 to Present ("Full")
full = modis8.filterBounds(region_of_interest).filterDate('2012-01-01', now).sort('system:time_start').map(maskClouds).map(add_ndvi).select('ndvi')
# 2014 to Present ("Ten")
ten = modis8.filterBounds(region_of_interest).filterDate('2014-01-01', now).sort('system:time_start').map(maskClouds).map(add_ndvi).select('ndvi')
# 2017 to Present ("Half")
half = modis8.filterBounds(region_of_interest).filterDate('2017-01-01', now).sort('system:time_start').map(maskClouds).map(add_ndvi).select('ndvi')

## GET REFERENCE/BASELINE DATA ##

# Reference NDVI for 2013 to 2022 for baseline
reference = full.filterDate('2013-01-01', '2022-12-31').filter(ee.Filter.calendarRange(121, 303, 'day_of_year')).sort('system:time_start')

# Compute the mean NDVI for the reference time frame
baseline_mean = reference.mean().clip(region_of_interest).updateMask(cropland)

# Visualization parameters for NDVI
vis = {'min': 1000, 'max': 5000, 'palette': ['brown', 'yellow', 'green']}

# Map NDVI
Map.addLayer(baseline_mean, vis, 'Baseline NDVI (Mean Value between 2013-2022)')

##############################################
### CALCULATE MEAN NDVI FOR RECENT YEARS ###
##############################################

# Recent datasets (2022, 2023, 2024)
recent_22 = full.filterDate('2022-01-01', '2022-12-31').filter(ee.Filter.calendarRange(121, 303, 'day_of_year'))
recent_23 = full.filterDate('2023-01-01', '2023-12-31').filter(ee.Filter.calendarRange(121, 303, 'day_of_year'))
recent_24 = full.filterDate('2024-01-01', now).filter(ee.Filter.calendarRange(121, 303, 'day_of_year'))

# Calculate recent mean
mean_recent_22 = recent_22.mean().clip(region_of_interest).updateMask(cropland)
mean_recent_23 = recent_23.mean().clip(region_of_interest).updateMask(cropland)
mean_recent_24 = recent_24.mean().clip(region_of_interest).updateMask(cropland)

##########################################
######### CALCULATING ANOMALIES ##########
##########################################

# Define function to subtract the reference mean
def subtract_mean(image):
    return image.subtract(baseline_mean).copyProperties(image, ['system:time_start'])

# Define function to convert to percentage
def percentage(image):
    return image.divide(baseline_mean).multiply(100).copyProperties(image, ['system:time_start'])

# Anomalies
anomalies_22 = recent_22.map(subtract_mean)
anomalies_23 = recent_23.map(subtract_mean)
anomalies_24 = recent_24.map(subtract_mean)

# Convert to percentage
anomalies_pct_22 = anomalies_22.map(percentage)
anomalies_pct_23 = anomalies_23.map(percentage)
anomalies_pct_24 = anomalies_24.map(percentage)

# Get the mean for the percent change for each pixel
anomalies_pct_mean_22 = anomalies_pct_22.mean()
anomalies_pct_mean_23 = anomalies_pct_23.mean()
anomalies_pct_mean_24 = anomalies_pct_24.mean()

# Combine anomalies from 2023 and 2024
combined_anomalies = ee.ImageCollection([
    anomalies_pct_mean_23,
    anomalies_pct_mean_24
])

# Calculate mean anomalies
mean_anomalies = combined_anomalies.mean()

#########################################
########## MAPPING ANOMALIES ############
#########################################

# Define visualization parameters
veganoms_vis = {
    'min': -50,
    'max': 50,
    'palette': ['darkred', 'red', 'yellow', 'green', 'darkgreen']
}

# Adding Outline to the Map
outline = region_of_interest.style(fillColor='00000000')

# Map the average anomalies for different years
# Map.addLayer(anomalies_pct_mean_22, veganoms_vis, 'Average anomaly Oct-Nov 2022')
# Map.addLayer(anomalies_pct_mean_23, veganoms_vis, 'Average anomaly Oct-Nov 2023')
# Map.addLayer(anomalies_pct_mean_24, veganoms_vis, 'Average anomaly Oct-Nov 2024')

# Add mean anomaly layer to the map
Map.addLayer(mean_anomalies, veganoms_vis, 'Mean Anomaly')
Map.addLayer(outline, {}, "Region of Interest")

# Create legend
Map.add_colorbar(
    veganoms_vis, label="Vegetation Index Anomaly Percentage", layer_name="Mean Anomaly", orientation="horizontal"
)

########################################
####### EXPORT IMAGES TO DRIVE #########
########################################

# Export Mean Anomalies
geemap.ee_export_image_to_drive(
    mean_anomalies, description='Mean_Anomalies', folder='export', region=admin1, scale=500, maxPixels=1000000000000
)

# Export Anomalies 2022
geemap.ee_export_image_to_drive(
    anomalies_pct_mean_22, description='Anomalies_22', folder='export', region=admin1, scale=500, maxPixels=1000000000000
)

# Export Anomalies 2023
geemap.ee_export_image_to_drive(
    anomalies_pct_mean_23, description='Anomalies_23', folder='export', region=admin1, scale=500, maxPixels=1000000000000
)

# Export Anomalies 2024
geemap.ee_export_image_to_drive(
    anomalies_pct_mean_24, description='Anomalies_24', folder='export', region=admin1, scale=500, maxPixels=1000000000000
)

# Display map
Map

Map(center=[14, 34], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(chil…